In [0]:
from pathlib import Path

from fastai import *
from fastai.basic_data import *
from fastai.text import *
from torch import *
import pandas as pd
from sklearn.model_selection import train_test_split

In [89]:
if torch.cuda.is_available():
  print("True")

True


In [0]:
with open('texts_train.txt', 'r') as file_texts:
    texts = file_texts.read().split("\n")
file_texts.close()

with open('scores_train.txt', 'r') as file_scores:
    scores = file_scores.read().split("\n")
file_scores.close()

In [0]:
texts_train, texts_val, scores_train, scores_val = train_test_split(texts, scores, train_size=0.85)

d_train = {'review_content': texts_train, 'rating': scores_train}
training_df = pd.DataFrame(data=d_train)

d_val = {'review_content': texts_val, 'rating': scores_val}
validation_df = pd.DataFrame(data=d_val)

In [92]:
validation_df

,review_content,rating
0,..когда-то в далеком 93 я купил две видеокассе...,10
1,Отличный образчик боевой фантастики. Особенно ...,9
2,"Потроясающе. Юмор, владение сюжетной линией. А...",9
3,Самая любимая книга у Коэльо. Паоло Коэльо как...,9
4,Нормально. Но желания сразу перечитать не возн...,7
...,...,...
2996,мои первые шаги в осознанность начинались вот ...,10
2997,"Кто знает об авторе, какие-нибудь сведения. Я ...",10
2998,"Как и ожидалось, книга захватывает. Герой заст...",9
2999,Надо было назвать этот фильм 4 идиота. А если ...,8


In [0]:
with open('dataset_40757_1.txt', 'r') as file_tests:
    texts_test = file_tests.read().strip("\n").split("\n")
file_tests.close()

d_test = {'review_content': texts_test}
test_df = pd.DataFrame(data=d_test)

In [94]:
test_df

,review_content
0,ну чото там все понятно было сразу.не очень ин...
1,Мне книга понравилась - очень - этакая смесь м...
2,В отношении книги меня можно записать в число ...
3,нет определённого мнения насчёт этого фильма. ...
4,"Тех кто считает, что «войны будут всегда, пока..."
...,...
995,"фильм показаллся устарелым: игра актеров, общи..."
996,"Я таки ""добила"" третью часть ""Берсерков"". Ради..."
997,Фэповско-нашистская подстава. И подмена. Делиш...
998,абсолютно двоякое чувство к этому фильму. хоче...


In [0]:
path =  Path('data')

# Language model data
data_lm = TextLMDataBunch.from_df(
    path,
    training_df, 
    valid_df=validation_df, 
    test_df=test_df, 
    text_cols=['review_content'], 
    label_cols=['rating'])

# Classifier model data
data_clas = TextClasDataBunch.from_df(
    path, 
    train_df=training_df, 
    valid_df=validation_df, 
    test_df=test_df, 
    vocab=data_lm.train_ds.vocab, 
    text_cols=['review_content'], 
    label_cols=['rating'], bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl')
data_clas = load_data(path, 'data_clas_export.pkl', bs=16)

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [99]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.928420,5.423859,0.228155,02:34


In [100]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.288733,4.911573,0.267385,02:58


In [0]:
learn.save_encoder('ft_enc')

In [102]:
learn.predict('да', n_words=10)

'да смешно . мне понравилось , но как будто стоит было'

In [103]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
# Load the language model we just trained as encoder
learn.load_encoder('ft_enc')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (17000 items)
x: TextList
xxbos xxmaj лучшая вещь у xxmaj акунина . xxmaj явно выделяется из прочих , которые , по - моему , стали чересчур похожи друг на друга . xxmaj из новелл лучшая , кажется , об английском кладбище .,xxbos xxmaj отличная книга ! xxmaj хотя я и не увлекаюсь фентези , но тут что - то меня xxunk прочитать эту книгу по многочисленным рекомендациям клуба . xxmaj не разачарована . xxmaj как пишут многие , действительно было искренне жаль дочитывать . xxmaj обычно я xxunk прочесть поскорее , узнать развязку , но тут с самого начала было ясно что добро победит зло :) и я наслаждалась слогом , юмором , доброй сказкой :) xxmaj хорошо что книги есть продолжения :) xxmaj замечательно,xxbos книга не произвела xxunk,xxbos xxmaj не ожидала от xxmaj xxunk такой слабой вещи . xxmaj идея отличная - клинки и все такое , но читала с xxunk ... xxmaj хотя , может это перевод такой дурацкий .,xxbos xxmaj xxunk супер личность . xxmaj 

In [104]:
data_clas.show_batch()

text,target
"xxbos xxmaj посмотрел тут "" 12 "" xxmaj михалкова . xxmaj начало просто завораживающее , с самогон начала на весь экран - отличная фраза "" xxmaj не пытайтесь найти правду быта , пытайтесь найти правду бытия "" ( тут же записал ) , снято отлично , прямо xxunk в экран , потом эти двенадцать мужиков , как они в xxunk xxunk - в общем , первые 20 минут наверное .",2
"xxbos xxmaj три брата – флегматик ( xxmaj xxunk ) , меланхолик ( xxmaj броуди ) и холерик ( xxmaj уилсон ) , – не xxunk год после гибели отца , отправляются в путешествие по xxmaj индии в вагоне первого класса в компании с xxunk розовыми чемоданами , xxunk , аппаратом для ламинирования и секретарем в очках для поддержания xxunk дня . xxmaj цель поездки – xxunk семейных xxunk и",8
"xxbos xxmaj роман xxmaj маргарет xxmaj митчел приоткрывает нам дверь в американское общество времен xxmaj гражданской войны . xxmaj войны xxmaj севера и xxmaj юга . xxmaj страшной войны . xxmaj ведь войны внутри страны самые страшные . xxmaj на фоне этих ужасных событий и разворачивается действие этого романа . xxmaj автор знакомит нас с очаровательной девушкой xxmaj скарлетт о""хара , забавная папина любимица , которая всегда получает то ,",10
"xxbos xxmaj однажды в прессу xxunk высказывание xxmaj чака xxmaj норриса , xxunk в порядке общей xxunk , о том , что из него такой же актёр , как из xxmaj дэвида xxmaj xxunk – спортсмен . и тем не менее встреча двух « звёзд » фильмов действия – верных последователей дела xxmaj брюса xxmaj ли , xxunk первую известность благодаря проектам , к которым был так или иначе причастен",6
"xxbos xxmaj если отобрать всё самое странное и xxunk , что только было в « трилогии xxunk , доведя это до наивысшей степени концентрации , – как раз получится очень необычное произведение со столь xxunk и xxunk названием . xxmaj горько xxunk в своих ожиданиях те , кто xxunk на восприятие научно - фантастической комедии . xxmaj или даже мрачной xxunk , в которой xxunk фантазия художника - визионера соседствует",6


In [105]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.831686,1.764436,0.331000,02:46


In [0]:
# Save intermediate model to enable resuming training from stages
learn.save('stage-1-model.pkl')

In [107]:
learn.load('stage-1-model.pkl')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (17000 items)
x: TextList
xxbos xxmaj лучшая вещь у xxmaj акунина . xxmaj явно выделяется из прочих , которые , по - моему , стали чересчур похожи друг на друга . xxmaj из новелл лучшая , кажется , об английском кладбище .,xxbos xxmaj отличная книга ! xxmaj хотя я и не увлекаюсь фентези , но тут что - то меня xxunk прочитать эту книгу по многочисленным рекомендациям клуба . xxmaj не разачарована . xxmaj как пишут многие , действительно было искренне жаль дочитывать . xxmaj обычно я xxunk прочесть поскорее , узнать развязку , но тут с самого начала было ясно что добро победит зло :) и я наслаждалась слогом , юмором , доброй сказкой :) xxmaj хорошо что книги есть продолжения :) xxmaj замечательно,xxbos книга не произвела xxunk,xxbos xxmaj не ожидала от xxmaj xxunk такой слабой вещи . xxmaj идея отличная - клинки и все такое , но читала с xxunk ... xxmaj хотя , может это перевод такой дурацкий .,xxbos xxmaj xxunk супер личность . xxmaj 

In [108]:
learn.freeze_to(-2)
# Slice up the learning rates to apply different rates to different layers of the network - the deeper the larger.
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.741259,1.672128,0.352000,03:36


In [0]:
learn.save('stage-2-model.pkl')

In [110]:
learn.load('stage-2-model.pkl')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (17000 items)
x: TextList
xxbos xxmaj лучшая вещь у xxmaj акунина . xxmaj явно выделяется из прочих , которые , по - моему , стали чересчур похожи друг на друга . xxmaj из новелл лучшая , кажется , об английском кладбище .,xxbos xxmaj отличная книга ! xxmaj хотя я и не увлекаюсь фентези , но тут что - то меня xxunk прочитать эту книгу по многочисленным рекомендациям клуба . xxmaj не разачарована . xxmaj как пишут многие , действительно было искренне жаль дочитывать . xxmaj обычно я xxunk прочесть поскорее , узнать развязку , но тут с самого начала было ясно что добро победит зло :) и я наслаждалась слогом , юмором , доброй сказкой :) xxmaj хорошо что книги есть продолжения :) xxmaj замечательно,xxbos книга не произвела xxunk,xxbos xxmaj не ожидала от xxmaj xxunk такой слабой вещи . xxmaj идея отличная - клинки и все такое , но читала с xxunk ... xxmaj хотя , может это перевод такой дурацкий .,xxbos xxmaj xxunk супер личность . xxmaj 

In [111]:
learn.freeze_to(-3)
# Slice up the learning rates to apply different rates to different layers of the network - the deeper the larger.
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.716906,1.628634,0.361000,04:22


In [0]:
learn.save('stage-3-model.pkl')

In [113]:
learn.load('stage-3-model.pkl')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (17000 items)
x: TextList
xxbos xxmaj лучшая вещь у xxmaj акунина . xxmaj явно выделяется из прочих , которые , по - моему , стали чересчур похожи друг на друга . xxmaj из новелл лучшая , кажется , об английском кладбище .,xxbos xxmaj отличная книга ! xxmaj хотя я и не увлекаюсь фентези , но тут что - то меня xxunk прочитать эту книгу по многочисленным рекомендациям клуба . xxmaj не разачарована . xxmaj как пишут многие , действительно было искренне жаль дочитывать . xxmaj обычно я xxunk прочесть поскорее , узнать развязку , но тут с самого начала было ясно что добро победит зло :) и я наслаждалась слогом , юмором , доброй сказкой :) xxmaj хорошо что книги есть продолжения :) xxmaj замечательно,xxbos книга не произвела xxunk,xxbos xxmaj не ожидала от xxmaj xxunk такой слабой вещи . xxmaj идея отличная - клинки и все такое , но читала с xxunk ... xxmaj хотя , может это перевод такой дурацкий .,xxbos xxmaj xxunk супер личность . xxmaj 

In [68]:
learn.predict("Плохо, но отвращения не вызывает, что уже хорошо. \
Если по сути, то сценарий - полный бред. Тут уже писали, что странно, \
что лилипуты, которые находятся еще в рыцарском веке, вдруг раз и построили \
современный дом для Гулливера, не говоря уже об электричестве.Ну ладно, может они такие просвещенные, \
что предпочитают беречь природу, а не засорять ее техногенными отходами. Но когда главный злодей умудрился сделать \
робота по инструкции из какого-то подросткового-нердовского журнала \"Как сделать робота\" на один разворот, это уже \
никак не объяснить вообще, кроме как пофигизма создателей фильма. Плюс к тому остается загадкой, как Гулливер и Дарси \
выбрались с острова. Туда они попали непонятно как тоже, ну если допустить, что смерч из бермудского треугольника выносит \
всех в него попавших на лилипутский остров, то можно жить. А куда он выкидывает людей, если они идут в обратном \
направлении? Прямо на остров Манхеттен? Что касается главного героя - то абсолютно несимпатичнй мне персонаж. \
Опять параллель с Иванушкой -Дурачком. Ничего особого не делал, никакими талантами не обладал, тут по \
счастливой случайности вляпался в историю, счастливым образом вышел из нее сухим и получил полкоралевства и царевну впридачу. \
А его товарищ, который работает и пытается совим горбом получить хоть какое-то повышение остается ни с чем. \
вообще не очень люблю такие сказки, потому что они запечатляют в детских головах четкий импринт, что \
много работать - плохо, а мало работать - хорошо. А отношения главных героев вообще вставили, наверное, \
потому что \"как же без них\", но абсолютно без души. Даже в подростковых комедиях типа \"Американского пирога\" \
пары подбирают куда лучше. Здесь же, при поцелуе не вздрагивает ни одна романтическая жилка. \
Добавили бы хоть какую-нибудь искорку.")

(Category 8,
 tensor(9),
 tensor([1.7086e-05, 1.6910e-02, 1.3306e-01, 1.7865e-02, 4.1020e-02, 8.0780e-02,
         1.2592e-01, 9.8397e-02, 1.3294e-01, 1.8242e-01, 1.7068e-01]))

In [69]:
learn.predict("Я не считаю что это лучшая книга -- скорее я бы так охарактеризовал одну из первых книг серии, но написано замечательно. \
Плюс отсылка к Куприну забавная. Плюс наконец то объясняет предысторию событий.")

(Category 10,
 tensor(2),
 tensor([2.2118e-05, 3.6257e-02, 2.7368e-01, 2.9112e-02, 3.7280e-02, 5.0455e-02,
         7.0328e-02, 5.2051e-02, 8.1394e-02, 1.4846e-01, 2.2096e-01]))

In [70]:
learn.predict("от первого фильма было странное чувство: когда на экране нет Михалкова, \
с трудом но смотришь.как только появляется - полный капец. посмотрев, по рекомендации Задорнова, \
второй фильм, понял в чем дело.образ комдива в этих продолжениях насквозь сжив. попытка скрестить Котовского с \
Рокоссовским отвратительна.такие как Котов,умеющие только рубать шашкой монахов, топить баржами пленных беляков да \
травить газами несчастных русских мужиков, сами в атаку сроду не \
ходили и талантами стратегов не обладали.и правильно Сталин таких Котовых порасстрелял перед войной.\
почитайте Суворова\"Очищение\"")

(Category 9,
 tensor(10),
 tensor([7.6083e-06, 2.2518e-02, 2.2999e-01, 1.7698e-02, 2.9450e-02, 4.6191e-02,
         6.1172e-02, 6.6724e-02, 1.0308e-01, 1.7741e-01, 2.4576e-01]))

In [114]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.663359,1.590418,0.374000,07:12


In [0]:
learn.save('final-model.pkl')

In [116]:
learn.load('final-model.pkl')

RNNLearner(data=TextClasDataBunch;

Train: LabelList (17000 items)
x: TextList
xxbos xxmaj лучшая вещь у xxmaj акунина . xxmaj явно выделяется из прочих , которые , по - моему , стали чересчур похожи друг на друга . xxmaj из новелл лучшая , кажется , об английском кладбище .,xxbos xxmaj отличная книга ! xxmaj хотя я и не увлекаюсь фентези , но тут что - то меня xxunk прочитать эту книгу по многочисленным рекомендациям клуба . xxmaj не разачарована . xxmaj как пишут многие , действительно было искренне жаль дочитывать . xxmaj обычно я xxunk прочесть поскорее , узнать развязку , но тут с самого начала было ясно что добро победит зло :) и я наслаждалась слогом , юмором , доброй сказкой :) xxmaj хорошо что книги есть продолжения :) xxmaj замечательно,xxbos книга не произвела xxunk,xxbos xxmaj не ожидала от xxmaj xxunk такой слабой вещи . xxmaj идея отличная - клинки и все такое , но читала с xxunk ... xxmaj хотя , может это перевод такой дурацкий .,xxbos xxmaj xxunk супер личность . xxmaj 

In [0]:
learn.predict("Сериал очень люблю, но Академия и Земля вызывает у меня отторжение идеей... \
Не люблю, когда принижают ценность человека как личности, даже не смотря на ошибки личности...")

In [117]:
for text in texts_test:
  print(learn.predict(text)[0])

9
9
10
5
10
9
10
10
10
9
10
7
9
5
8
10
9
10
10
10
2
10
9
9
10
3
9
8
5
9
10
9
10
9
10
10
7
9
9
10
10
9
7
8
10
8
7
10
10
10
10
10
8
8
5
9
10
10
9
10
9
9
9
9
10
8
10
9
10
10
7
8
8
10
8
9
7
10
8
8
8
10
8
10
8
7
8
9
9
8
10
5
9
9
10
8
7
9
10
10
10
9
9
8
8
8
8
10
9
10
10
8
8
10
10
9
8
10
10
9
7
8
10
9
10
9
10
10
9
10
9
10
8
8
8
10
5
9
10
10
8
7
10
9
9
10
9
8
7
10
9
10
9
10
10
10
5
8
9
10
10
7
10
10
9
8
9
10
8
6
7
8
5
10
5
10
10
10
10
10
8
9
9
7
10
5
8
9
10
8
8
9
5
10
9
5
10
9
9
9
7
8
10
10
9
6
8
10
9
8
9
10
10
10
7
9
8
4
9
10
10
10
8
8
10
9
10
9
10
10
10
10
10
8
9
9
8
9
9
9
9
10
10
10
10
7
10
9
5
10
8
8
9
9
10
10
10
7
5
9
9
9
9
8
10
8
10
8
9
8
10
8
9
9
10
9
10
9
8
8
5
8
9
10
10
7
10
10
9
9
10
9
10
9
7
10
5
8
9
9
9
9
9
5
8
9
10
10
10
10
7
8
8
10
10
8
10
10
9
8
9
10
10
9
10
10
10
9
10
9
7
4
9
9
8
10
7
10
5
8
10
10
10
10
8
8
9
9
8
9
10
9
10
9
9
5
10
7
10
9
9
10
10
10
10
8
10
7
10
10
8
10
10
9
8
7
10
9
8
9
10
9
10
9
10
8
9
10
7
8
10
9
10
7
10
8
10
10
9
9
10
9
8
8
10
8
9
9
10
9
9
8
10
10
9
8
9
10
